In [ ]:
# Add project src to path.
import set_path

import numpy as np
import logging

import src.region_growing as growing
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils
from src.labels import Labels
from src.pipeline import Pipeline

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

In [ ]:
tilecode = '2386_9702'
# tilecode = '2397_9705'

in_file = '../datasets/pointcloud/processed_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/refined_' + tilecode + '.laz'

target_labels = [Labels.BUILDING, Labels.CAR, Labels.TREE, Labels.TRAFFIC_SIGN, Labels.STREET_LIGHT, Labels.CITY_BENCH, Labels.RUBBISH_BIN]
exclude_labels = (Labels.NOISE,)

ahn_data_folder = '../datasets/ahn/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder, caching=False)

In [ ]:
params = {'top': 1., 'grid_size': 0.4, 'min_comp_size': 100}

refiner = growing.RefineGround(target_labels, npz_reader, params)

---
"Manual" processing of the tile.

In [ ]:
pointcloud = las_utils.read_las(in_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

In [ ]:
mask = np.ones((len(points),), dtype=bool)
for l in exclude_labels:
    mask = mask & (labels != l)

In [ ]:
# TODO: why are no points added?

In [ ]:
ref_mask = refiner.get_label_mask(points, labels, mask, tilecode)

In [ ]:
for label, mask in zip(target_labels, ref_mask):
    labels[mask] = label

In [ ]:
las_utils.label_and_save_las(pointcloud, labels, out_file)

---
Add it to the pipeline.

**DOESN'T WORK YET**

In [ ]:
# Set-up region growing based on PCL pipeline.
process_sequence = (refiner,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

In [ ]:
pipeline.process_file(in_file, out_file=out_file)